# utils 

> utils for this project

In [ ]:
# | default_exp utils

In [ ]:
# | hide
# from nbdev.showdoc import *

## parsing dependencies

By default nbdev uses `requirements` and `dev_requirements` attributes form `settings.ini` to install dependencies.

However, we want to use `Pipfile` file to install dependencies.
 
To do so, we need to override the default behavior of nbdev. We do this by creating a custom `DependencyParser` class.

 Example of setuptools expected format for dependencies:
 
 ```python
     install_requires=[
        'docutils',
        'BazSpam ==1.1',
        "black['jupyter'] ==22.12.0",
    ],
```

In [ ]:
# | export

import toml


class DependencyParser:
    """parse a pipfile to get the requirements

    Parameters
    ----------
    path_to_pipfile : str
        path to the pipfile

    """

    def __init__(self, path_to_pipfile: str):
        self.path_to_pipfile = path_to_pipfile
        with open(path_to_pipfile) as f:
            self.pipfile = toml.load(f)

    def __repr__(self):
        return f"DependencyParser({self.path_to_pipfile})"

    @staticmethod
    def pipenv_to_setuptools(package_name: str, contraints: str | dict) -> str:
        if contraints == "*":
            res = f"{package_name}"
        elif (contraints != "*") and isinstance(contraints, str):
            res = f"{package_name}{contraints}"
        elif isinstance(contraints, dict):
            if extras := contraints.get("extras"):
                extras = [f"'{e}'" for e in extras]  # add ' to extras in list
                package_name = f"{package_name}[{','.join(extras)}]"
                res = f"{package_name}{contraints['version']}"
            else:
                res = f"{package_name}{contraints['version']}"
        else:
            raise ValueError(f"Could not parse {package_name}, {contraints}")

        return res

    @property
    def min_python(self) -> str:
        return self.pipfile["requires"]["python_version"]

    @property
    def requirements(self):
        return [
            self.pipenv_to_setuptools(k, v) for k, v in self.pipfile["packages"].items()
        ]

    @property
    def dev_requirements(self):
        return [
            self.pipenv_to_setuptools(k, v)
            for k, v in self.pipfile["dev-packages"].items()
        ]

    def dump_requirements(self, path: str):
        import json

        with open(path, "w") as f:
            reqs = {
                "_comment": "# WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!",
                "min_python": self.min_python,
                "requirements": self.requirements,
                "dev_requirements": self.dev_requirements,
            }
            json.dump(reqs, f, indent=4)

In [ ]:
# test DependencyParser on test data
deps_parsed = DependencyParser("../test_data/utils/Pipfile")
assert deps_parsed.min_python == "3.6"
assert set(deps_parsed.requirements) == set(["simple==2.28"])
assert set(deps_parsed.dev_requirements) == set(
    ["nested['jupyter']==22.12.0", "flexible==1.*", "star", "gt>2.20", "gte>=2.20"]
)

In [ ]:
# | hide
import json

test_config_path = "../test_data/utils/setup.json"
deps_parsed.dump_requirements(test_config_path)

# test setup.config
with open(test_config_path) as f:
    d = json.load(f)

assert d == {
    "_comment": "# WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT!",
    "min_python": "3.6",
    "requirements": ["simple==2.28"],
    "dev_requirements": [
        "nested['jupyter']==22.12.0",
        "flexible==1.*",
        "star",
        "gt>2.20",
        "gte>=2.20",
    ],
}

In [ ]:
# get dependencies from setup.json
with open("../setup.json") as f:
    deps = json.load(f)

min_python = deps["min_python"]
requirements = deps["requirements"]
dev_requirements = deps["dev_requirements"]

In [ ]:
type(dev_requirements)

list

In [ ]:
{"dev": dev_requirements}

{'dev': ["black['jupyter'] ==22.12.0",
  'blacken-docs ==1.12.1',
  'isort ==5.10.1',
  'jupyter ==1.*',
  'mypy',
  'nbdev ==2.3.9',
  'nbqa ==1.5.3',
  'pre-commit ==2.20',
  'types-requests',
  'types-toml']}

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()